In [2]:
import pandas as pd

df_crims = pd.read_csv('CMX_2024.csv')

display(df_crims.head())

,anio_inicio,mes_inicio,fecha_inicio,hora_inicio,anio_hecho,mes_hecho,fecha_hecho,hora_hecho,delito,categoria_delito,...,fiscalia,agencia,unidad_investigacion,colonia_hecho,colonia_catalogo,alcaldia_hecho,alcaldia_catalogo,municipio_hecho,latitud,longitud
0,2024,Enero,2024-01-01,00:20:00,2024.0,Enero,2024-01-01,00:20:00,PERDIDA DE LA VIDA POR OTRAS CAUSAS,HECHO NO DELICTIVO,...,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN TLALPAN,UAT-TLP-3,UI-1SD,SAN MIGUEL TOPILEJO,Pueblo San Miguel Topilejo,TLALPAN,Tlalpan,CDMX,19.198611,-99.140064
1,2024,Enero,2024-01-01,01:12:00,2023.0,Diciembre,2023-12-28,21:45:00,HOMICIDIO CULPOSO POR TRÁNSITO VEHICULAR (COLI...,DELITO DE BAJO IMPACTO,...,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN AZCAP...,UAT-AZ-4,UI-2SD,AERONÁUTICA MILITAR,Aeronautica Militar,VENUSTIANO CARRANZA,Venustiano Carranza,CDMX,19.420629,-99.115402
2,2024,Enero,2024-01-01,01:14:00,2023.0,Diciembre,2023-12-31,23:00:00,PERDIDA DE LA VIDA POR CAIDA,HECHO NO DELICTIVO,...,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN IZTAP...,UAT-IZP-5,1 SIN DETENIDO,OJITO DE AGUA,Pueblo Santa Maria Aztahuacan,IZTAPALAPA,Iztapalapa,CDMX,19.347257,-99.024526
3,2024,Enero,2024-01-01,02:29:00,2023.0,Diciembre,2023-12-31,23:50:00,"ROBO EN EVENTOS MASIVOS (DEPORTIVOS, CULTURALE...",DELITO DE BAJO IMPACTO,...,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN CUAUH...,UAT-CUH-6,UI-1SD,CENTRO,Centro,CUAUHTEMOC,Cuauhtémoc,CDMX,19.434052,-99.134837
4,2024,Enero,2024-01-01,03:26:00,2024.0,Enero,2024-01-01,02:10:00,PERDIDA DE LA VIDA POR PARO CARDIACO,HECHO NO DELICTIVO,...,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN MAGDA...,UAT-MC-2,UI-1SD,LOMAS DE SAN BERNABÉ,Lomas De San Bernabe,LA MAGDALENA CONTRERAS,La Magdalena Contreras,CDMX,19.314159,-99.263989


In [3]:
from datetime import datetime
import nltk

# Convert 'fecha_hecho' to datetime
df_crims['fecha_hecho'] = pd.to_datetime(df_crims['fecha_hecho'])

df_last_year = df_crims[df_crims['fecha_hecho'].dt.year == 2024]
df_delict = df_last_year[df_last_year['categoria_delito'] != 'HECHO NO DELICTIVO']

df_bajo_impacto = df_delict[df_delict['categoria_delito'] == 'DELITO DE BAJO IMPACTO']
small_delicts = df_bajo_impacto['delito'].value_counts()
filtered_delicts = small_delicts[small_delicts.index.str.contains('ROBO|ACOSO|HOMICICIO|PANDILLA', case=False)]

display(df_delict.shape)
# Filter out the delicts that are NOT in the filtered_delicts list
df_delict = df_delict[~((df_delict['categoria_delito'] == 'DELITO DE BAJO IMPACTO') & (~df_delict['delito'].isin(filtered_delicts.index)))]
display(df_delict.shape)

df_delict = df_delict.dropna(subset=['latitud', 'longitud'])
display(df_delict.shape)


#df_crimes = df_delict[['delito', 'longitud', 'latitud']]

display(df_delict['categoria_delito'].value_counts())

(119666, 21)

(41007, 21)

(38989, 21)

categoria_delito
DELITO DE BAJO IMPACTO                                     27610
ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA        4125
ROBO DE VEHÍCULO CON Y SIN VIOLENCIA                        3355
ROBO A NEGOCIO CON VIOLENCIA                                 993
VIOLACIÓN                                                    853
ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA        542
HOMICIDIO DOLOSO                                             481
ROBO A REPARTIDOR CON Y SIN VIOLENCIA                        358
ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA      196
LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO                190
ROBO A CASA HABITACIÓN CON VIOLENCIA                         120
ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA       76
ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA                 72
ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA                      12
SECUESTRO                                                      6
Name: co

In [4]:
crime_category_to_weight = {
    'DELITO DE BAJO IMPACTO': 1,
    'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA': 3,
    'ROBO DE VEHÍCULO CON Y SIN VIOLENCIA': 2,
    'ROBO A NEGOCIO CON VIOLENCIA': 3,
    'VIOLACIÓN': 4,
    'ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA': 2,
    'HOMICIDIO DOLOSO': 5,
    'ROBO A REPARTIDOR CON Y SIN VIOLENCIA': 3,
    'ROBO A PASAJERO A BORDO DE MICROBÚS CON Y SIN VIOLENCIA': 2,
    'LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO': 4,
    'ROBO A CASA HABITACIÓN CON VIOLENCIA': 3,
    'ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA': 3,
    'ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA': 3,
    'ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA': 2,
    'SECUESTRO': 5
}

df_crimes_with_weight = df_delict.copy()
df_crimes_with_weight['weight'] = df_crimes_with_weight['categoria_delito'].map(crime_category_to_weight)
df_crimes_with_weight['weight'] = df_crimes_with_weight['weight'].fillna(1)

display(df_crimes_with_weight['weight'].value_counts())

weight
1.0    27806
3.0     5744
2.0     3909
4.0     1043
5.0      487
Name: count, dtype: int64

In [ ]:
df_ponderado = df_crimes_with_weight[['delito', 'categoria_delito', 'longitud', 'latitud', 'weight']]
df_ponderado.loc[df_ponderado['delito'].str.contains('CON', case=True), 'weight'] += 0.5

df_weights = df_ponderado[['latitud','longitud','weight']]

df_weights['weight'] = df_weights['weight'].apply(lambda x: int(x)/5)
display(df_weights['weight'].value_counts())

C:\Users\11ser\AppData\Local\Temp\ipykernel_17720\3263705237.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weights['weight'] = scaler.fit_transform(df_weights[['weight']]).flatten()


weight
0.000    25016
0.625     4810
0.250     3155
0.125     2790
0.750     1020
0.500      934
0.375      754
1.000      487
0.875       23
Name: count, dtype: int64

In [60]:
import geojson
from geojson import Feature, FeatureCollection, Point

def df_to_geojson(df):
    features = []
    for _, row in df.iterrows():
        if not pd.isna(row['latitud']) and not pd.isna(row['longitud']):
            point = Point((row['longitud'], row['latitud']))
            features.append(
                Feature(geometry=point, properties={"weight": row['weight']})
            )
    return FeatureCollection(features)

geojson_data = df_to_geojson(df_weights)

# Save to a GeoJSON file
with open('crimes.geojson', 'w') as f:
    geojson.dump(geojson_data, f)

KeyboardInterrupt: 

In [66]:
def filter_geojson_by_coordinates(geojson_data, lat_min, lat_max, lon_min, lon_max):
    filtered_features = []
    for feature in geojson_data['features']:
        lon, lat = feature['geometry']['coordinates']
        if lat_min <= lat <= lat_max and lon_min <= lon <= lon_max:
            filtered_features.append(feature)
    
    return geojson.FeatureCollection(filtered_features)

# Example usage
lat_min, lat_max = 19.0, 20.0
lon_min, lon_max = -100.0, -98.0
filtered_geojson_data = filter_geojson_by_coordinates(geojson_data, lat_min, lat_max, lon_min, lon_max)

# Save the filtered GeoJSON to a new file
with open('filtered_crimes.geojson', 'w') as f:
    geojson.dump(filtered_geojson_data, f)

In [68]:
import folium

# Crear un mapa centrado en el área filtrada
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Añadir el archivo GeoJSON al mapa
folium.GeoJson('filtered_crimes.geojson').add_to(m)

# Mostrar el mapa


In [ ]:
import folium

def visualize_map(df):
    # Create a map centered around the average latitude and longitude
    m = folium.Map(location=[df['latitud'].mean(), df['longitud'].mean()], zoom_start=12)
    
    # Add points to the map
    for _, row in df.iterrows():
        if not pd.isna(row['latitud']) and not pd.isna(row['longitud']):
            folium.Marker(
                location=[row['latitud'], row['longitud']],
                popup=row['delito'],
                tooltip=row['categoria_delito']
            ).add_to(m)
    
    return m

# Example usage
map_visualization = visualize_map(df_crims)
map_visualization